### Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np
from tqdm import tqdm
from statsmodels.discrete.discrete_model import Logit
import statsmodels.api as sm
from scipy.optimize import minimize

## Problem 1: Dynamic Optimization 

In [2]:
data = pd.read_csv(r"C:\Users\Popov\Documents\Studies\NES_studies\Python\EIO\HA2\RUST_DATA.csv")

In [3]:
data.head(4)

,0,level_1,state,decision
0,4403,1983-05-01,0.0,0.0
1,4403,1983-06-01,0.0,0.0
2,4403,1983-07-01,1.0,0.0
3,4403,1983-08-01,2.0,0.0


### Task 1: Static model

In [4]:
# select data
X = data['state']
X = sm.add_constant(X)
Y = data['decision'].astype(int)

# fit logit regression
logit = Logit(Y, X).fit(cov_type="hc2")
print(logit.summary())

Optimization terminated successfully.
         Current function value: 0.036576
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:               decision   No. Observations:                 8260
Model:                          Logit   Df Residuals:                     8258
Method:                           MLE   Df Model:                            1
Date:                Wed, 16 Oct 2024   Pseudo R-squ.:                  0.1377
Time:                        14:07:17   Log-Likelihood:                -302.12
converged:                       True   LL-Null:                       -350.35
Covariance Type:                  hc2   LLR p-value:                 9.139e-23
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.3942      0.285    -25.917      0.000      -7.953      -6.835
state          0.0712      0

In [5]:
# Get the LaTeX output
latex_output = logit.summary().as_latex()

with open("logits.tex", "w") as f:
    f.write(latex_output)

### Task 2: Dynamic model

#### Transitions probabilities

In [6]:
# define difference variable - changes in states (mileage groups)
data['state_discrete'] = data['state'] - data['state'].shift(1)

# define the total sums of the changes (transitions)
N = data['state_discrete'].notna().sum().sum()

# finally, define the number of transitions into particular states
t1 = (data['state_discrete'] == 0).sum().sum() / N
t2 = (data['state_discrete'] == 1).sum().sum() / N
t3 = (data['state_discrete'] == 2).sum().sum() / N

# print them
rounded_probs = (round(t1, 3), round(t2, 3), round(t3, 3))
print(rounded_probs)

(0.343, 0.626, 0.011)


#### Transition matrix

In [7]:
# determine the number of states
T = max(data['state'].astype(int))+1 # add one, as python won't see the last value

# crete an empty transition matrix
P = np.zeros((T, T))

# add probabilities of staying in the same state
np.fill_diagonal(P, p1)

# add probability of getting into the next state
P = P + np.diag(np.ones(T - 1) * p2, 1)

# and probability of getting 2 states ahead
P = P + np.diag(np.ones(T - 2) * p3, 2)

# adjust the last column to ensure row sums equal 1
P[:, -1] += 1 - P.sum(1)

# see the matrix
print(P)

NameError: name 'p1' is not defined